# Preparation of Flux-Free Kitaev Honeycomb lattice state
## In HONEYCOMB LATTICE SPACE
2 honeycombs

N = 14 spins

Hamiltonian: $H = -J_x \sum_{\langle i,j \rangle_x} \sigma_i^x \sigma_j^x
    -J_y \sum_{\langle i,j \rangle_y} \sigma_i^y \sigma_j^y
    -J_z \sum_{\langle i,j \rangle_z} \sigma_i^z \sigma_j^z$ 

Step 1: prepare product state $|00..>$ of N spins

In [134]:
import numpy as np
import scipy
from scipy import sparse

In [135]:
import exact_diagonalization as ed
import importlib
importlib.reload(ed)

<module 'exact_diagonalization' from '/Users/giovanniconcheri/Desktop/TESI/MasterThesis/Exact_Diagonalization/exact_diagonalization.py'>

We are doing this mapping: 

$\ket{\uparrow \uparrow} = \ket{00} , \quad\ket{\downarrow \downarrow} = \ket{11} $

HONEYCOMB SITES
<div align="center">
    <img src="figures/Honeycomb_Mapping.jpeg" alt="Honeycomb Mapping" width="300"/>
</div>

TORIC CODE SITES (used in the functions to identify representative qubits and so on)
<div align="center">
    <img src="figures/Toric_Code_Sites.png" alt="Honeycomb Mapping" width="300"/>
</div>


In [136]:
n_spins = 14
e1 = np.array([1,0])
psi = e1
for _ in range(n_spins-1):
    psi = np.kron(psi,e1)

print(psi.shape)


assert psi.shape[0]== 2**14

(16384,)


In [137]:
print(psi)

[1 0 0 ... 0 0 0]


Identify a representative qubit in each plaquette: we identify qubit 0 and 4 (in the toric code space, i.e. toric spin 0 -> honeycomb spin (0,1) and so on) -> look notes  

We create operator that applies hadamard only on these two representative qubits

In [138]:
#representative qubits
qubit1 = 0
qubit2 = 4

In [139]:
n_eff_spins = int(n_spins/2)
H = ed.Hadamard(qubit1,qubit2,n_eff_spins)
print(H.shape)
psi1 = H @ psi.copy()

print(psi1.shape)

# Print non-zero values and their positions
non_zero_indices = np.nonzero(psi1)[0]  # Get the indices of non-zero values
non_zero_values = psi1[non_zero_indices]  # Get the non-zero values

print("Positions of non-zero values:", non_zero_indices)
print("Non-zero values of psi1:", non_zero_values)

(16384, 16384)
(16384,)
Positions of non-zero values: [    0    48 12288 12336]
Non-zero values of psi1: [0.5 0.5 0.5 0.5]


Create op. that applies CNOT on each plaquette with representative qubits as control qubits and all other qubits of plaquette as target ones

In [140]:
CNOT_op = ed.CNOT(qubit1, qubit2, n_eff_spins, 4)
#print(CNOT_op.toarray())
psi2 =  CNOT_op @ psi1.copy()
#print(psi1.shape)
#print(CNOT_op.shape)
#print(psi2.shape)
print(psi2)

# Print non-zero values and their positions
non_zero_indices = np.nonzero(psi2)[0]  # Get the indices of non-zero values
non_zero_values = psi2[non_zero_indices]  # Get the non-zero values

print("Positions of non-zero values:", non_zero_indices)
print("Non-zero values of psi2:", non_zero_values)

[0.5 0.  0.  ... 0.  0.  0. ]
Positions of non-zero values: [    0   255 16191 16320]
Non-zero values of psi2: [0.5 0.5 0.5 0.5]


Next step: apply $U_a$ to horizontal sites: effective qubit = 0,2,4,5, $U_b$ to vertical ones: effective qubit =1,3,6  $U = \prod_{vert} U_a \prod_{hor} U_b$

In [141]:
psi_fluxfree = ed.U_a_full(n_eff_spins) @ ed.U_b_full(n_eff_spins) @ psi2.copy()
print(psi_fluxfree.shape)
print(psi_fluxfree)
#psi_fluxfree is flux free state with ZZ bond coupling

(16384,)
[0.08838835+0.08838835j 0.        +0.j         0.        +0.j         ...
 0.        +0.j         0.        +0.j         0.        +0.j        ]


In [142]:
# Id = sparse.csr_array(np.eye(4))

# X = np.zeros((4,4))
# X[0,3] = 1.
# X[3,0] = 1.
# X = sparse.csr_array(X)

# Z = np.zeros((4,4))
# Z[0,0] = 1.
# Z[3,3] = -1.
# Z = sparse.csr_array(Z)

# Y = np.zeros((4,4))
# Y[0,1] = -1.j
# Y[1,0] = 1.j
# Y[2,3] = -1.j
# Y[3,2] = 1.j
# Y = sparse.csr_array(Y)

In [143]:
X = sparse.csr_array([[0.,1.],[1.,0.]])
Y = sparse.csr_array([[0.,-1.j],[1.j,0.]])
Z = sparse.csr_array([[1.,0.],[0.,-1.]])
I = sparse.csr_array(np.eye(2))

Now we create plaquette term!

In [144]:
Op_list = [X, Y, Z, X, Y, Z]
index_list = [3, 2, 1, 6, 7, 4]

W_tilde_1 = ed.Op_string(Op_list, index_list, I, n_spins)

# Compute the expectation value
expectation_value = psi_fluxfree.conj() @ W_tilde_1 @ psi_fluxfree

# Print the result
print("First plaquette term:", expectation_value)

First plaquette term: (0.9999999999999991-7.745036314193294e-20j)


In [145]:
Op_list = [X, Y, Z, X, Y, Z]
index_list = [7, 6, 9, 12, 13, 10]

W_tilde_2 = ed.Op_string(Op_list, index_list, I, n_spins)

# Compute the expectation value
expectation_value = psi_fluxfree.conj() @ W_tilde_2 @ psi_fluxfree

# Print the result
print("Second plaquette term:", expectation_value)

Second plaquette term: (0.9999999999999991-4.265443305812136e-18j)


We finally get the plaquette terms to be +1! So now we can apply a floquet drive to the flux free state. Note that the plaquette term commutes with the floquet time evolution, which means that it is a conserved quantity!

Before time evolving, let's do one last check. We check that the initial flux free state is an eigenstate of the ZZ terms on the vertical bonds. This means that the fermions are aligned on the vertical bonds, with one Majorana per site.

In [146]:
ZZ_1 = ed.Op_full(Z, I, [2,3], n_spins)
ZZ_2 = ed.Op_full(Z, I, [6,7], n_spins)
ZZ_3 = ed.Op_full(Z, I, [12,13], n_spins)
ZZ_other1 = ed.Op_full(Z, I, [0,1], n_spins)
ZZ_other2 = ed.Op_full(Z, I, [4,5], n_spins)

ZZ1Value = psi_fluxfree.conj() @ ZZ_1 @ psi_fluxfree
ZZ2Value = psi_fluxfree.conj() @ ZZ_2 @ psi_fluxfree
ZZ3Value = psi_fluxfree.conj() @ ZZ_3 @ psi_fluxfree
ZZother1Value = psi_fluxfree.conj() @ ZZ_other1 @ psi_fluxfree
ZZother2Value = psi_fluxfree.conj() @ ZZ_other2 @ psi_fluxfree

print("ZZ1Value:", ZZ1Value)
print("ZZ2Value:", ZZ2Value)
print("ZZ3Value:", ZZ3Value)
print("ZZother1Value:", ZZother1Value)
print("ZZother2Value:", ZZother2Value)

ZZ1Value: (0.9999999999999991+2.2018113140169403e-19j)
ZZ2Value: (0.9999999999999991+2.2018113140169403e-19j)
ZZ3Value: (0.9999999999999991+2.2018113140169403e-19j)
ZZother1Value: (0.9999999999999991+2.2018113140169403e-19j)
ZZother2Value: (0.9999999999999991+2.2018113140169403e-19j)


### Floquet drive

Ops $e^{i \frac{\pi}{4} \alpha \alpha} $ with $\alpha = X, Y, Z$

In [147]:
from scipy.sparse.linalg import expm_multiply, expm


In [148]:
psi_t, _ = ed.floquet_evolution(psi_fluxfree, 1, n_spins, ZZ = ZZ_2)
print(psi_t.shape)
#print(psi_t)

i,j,z 1 1 1
(16384,)


Let us now recheck the plaquette terms! (they should still be +1)

In [149]:
# Compute the complex conjugate transpose of psi_fluxfree
psi_t_dagger = psi_t.conj()

# Compute the expectation value 1
expectation_value1 = psi_t_dagger @ W_tilde_1 @ psi_t

# Print the result
print("First plaquette term:", expectation_value1)


# Compute the expectation value 2
expectation_value2 = psi_t_dagger @ W_tilde_2 @ psi_t

# Print the result
print("Second plaquette term:", expectation_value2)

First plaquette term: (0.9999999999999999+1.2037062152420224e-34j)
Second plaquette term: (0.9999999999999999-1.4444474582904269e-34j)


Now let us check how the fermion on the central vertical bond changes over floquet drive, i.e. let us calculate the expectation value ZZ on the central vertical bond (spins 6,7) and see how it changes. we expect it to oscillate between +1 and 0 per floquet cycle 

$\textbf{Why?}$

In [150]:
n_cycles = 5

psinew, list_zz = ed.floquet_evolution(psi_fluxfree, n_cycles, n_spins, ZZ = ZZ_2)

print(list_zz)

i,j,z 1 1 1
i,j,z 1 1 1
i,j,z 1 1 1
i,j,z 1 1 1
i,j,z 1 1 1
[np.complex128(0.9999999999999991+2.2018113140169403e-19j), np.complex128(6.071532165918825e-18-1.5377782402304176e-36j), np.complex128(0.9999999999999998+1.1131085005440987e-52j), np.complex128(-2.0816681711721685e-17+5.132699599308394e-35j), np.complex128(1.0000000000000002-1.3153578005405241e-20j), np.complex128(-1.457167719820518e-16+1.385733376431204e-34j)]


Now we go back to psi_fluxfree, which is a flux free state with fermions paired vertically, and we want to rearrange them such that they are paired diagonally (fig. 4b thesis paper https://arxiv.org/pdf/2501.18461). We do so by applying the YY evolution to the sites on the low part of the hexagon connected by the y-y bonds, and then the XX evolution to the "low" sites connected by XX bonds, as shown in figure

 ![Image](figures/M-Swap_gates.png)

$\textbf{Why can't we drive the YY-XX also in the upper part of the honeycomb}$?

We don't apply the evolutions to also the upper sites otherwise instead of having diagonal fermions weird things happen, you can convince yourself by doing the diagrams by hand on paper

In [151]:
psi_diagonal1 = ed.floquet_unitary(psi_fluxfree, n_spins, applyX = False, applyY = True, applyZ = False, Yindexlist = [[3,4],[7,10]])
psi_diagonal2 = psi_diagonal1.copy()
psi_diagonal2 = ed.floquet_unitary(psi_diagonal2, n_spins, applyX = True, applyY = False, applyZ = False, Xindexlist = [[4,7],[10,13]])
psi_diagonal = psi_diagonal2.copy()

i,j,z 0 1 0
i,j,z 1 0 0


Now let us check if this state is eigenstate with eigval = +1 of the ZXZX string acting on the sites [2,3,4,7]

In [69]:
ZXZX = ed.Op_string([Z, X, Z, X], [2, 3, 4, 7], I, n_spins)

# Compute the complex conjugate transpose of psi_diagonal
psi_diagonal_dagger = psi_diagonal.conj()

ZXZX_Value = psi_diagonal_dagger @ ZXZX @ psi_diagonal

print("ZXZX Value:", ZXZX_Value)


ZXZX Value: (0.9999999999999999-1.7132356737011346e-19j)


### e-m anyons

Now we want to create an m anyon at the center of the first plaquette. To do so we start from state psi_diagonal, and we apply a Z operator to the first plaquette -> new state has plaquette term Wp = -1 and a diagonal unoccupied fermion, i.e. m anyon!

In [70]:
psi_m = ed.Op_full(Z, I, 2, n_spins) @ psi_diagonal.copy()

#let us check if we actually have e anyon:

#check fermion occupation number
ZXZX_Value = psi_m.conj() @ ZXZX @ psi_m
print("ZXZX Value:", ZXZX_Value) #should be +1 (fermionic parity)

#check plaquette term:
W1_value = psi_m.conj() @ W_tilde_1 @ psi_m
print("First plaquette term:", W1_value) #should be -1



ZXZX Value: (0.9999999999999999-1.7132356737011346e-19j)
First plaquette term: (-0.9999999999999998-3.204200040492813e-19j)


Given m anyon, now we apply floquet drive to transmute into e anyon!

In [71]:
psi_e = ed.floquet_unitary(psi_m, n_spins, applyX = True, applyY = True, applyZ = True)

#let us check if we actually have e anyon:

#check fermionic parity
ZXZX_Value = psi_e.conj() @ ZXZX @ psi_e
print("ZXZX Value:", ZXZX_Value) #should be -1

#check plaquette term:
W1_value = psi_e.conj() @ W_tilde_1 @ psi_e
print("First plaquette term:", W1_value) #should be -1

i,j,z 1 1 1
ZXZX Value: (-0.9999999999999993+7.703719777548943e-33j)
First plaquette term: (-0.9999999999999996+3.851859888774472e-34j)


As a last check, let us measure  an electric loop around the plaquette. It should correspond to +1 for an e anyon, and to -1 for an m anyon. Therefore, driving the system with Floquet drive, it should oscillate between +-1

In [72]:
Op_list = [Z, X, Z, X]
index_list = [1, 2, 7, 6]

e_loop = ed.Op_string(Op_list, index_list, I, n_spins)

# Compute the expectation value
expectation_value_e = psi_e.conj() @ e_loop @ psi_e
# Print the result
print("e anyon: e loop term:", expectation_value_e) #should be +1

# Compute the expectation value
expectation_value_m = psi_m.conj() @ e_loop @ psi_m
# Print the result
print("m anyon: e loop term:", expectation_value_m) #should be -1

#for now it does the opposite of what it should do :)


e anyon: e loop term: (0.9999999999999993-6.162975822039155e-33j)
m anyon: e loop term: (-0.9999999999999999+6.143054205688061e-19j)
